In [7]:
!git clone https://github.com/mlflow/mlflow.git

Cloning into 'mlflow'...
remote: Enumerating objects: 51167, done.
remote: Total 51167 (delta 0), reused 0 (delta 0), pack-reused 51167
Receiving objects: 100% (51167/51167), 69.57 MiB | 939.00 KiB/s, done.
Resolving deltas: 100% (39229/39229), done.


In [5]:
import os
import warnings
import sys
import argparse

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [8]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [9]:
np.random.seed(40)

In [19]:
# alpha=0.5
parser = argparse.ArgumentParser()
parser.add_argument("--alpha")
parser.add_argument("--l1-ratio")
args = parser.parse_args("")

In [13]:
# Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
wine_path = "wine-quality.csv"
data = pd.read_csv(wine_path)

In [14]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

In [15]:
# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [23]:
alpha = float(0.5)
l1_ratio = float(0.5)

In [24]:
with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160691
  R2: 0.12678721972772689


/opt/anaconda3/envs/AIF/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [28]:
# !mlflow models build-docker \
#     -m /Users/macbook/Documents/GitHub/mlflow/wine_quality/mlruns/0/6d0e391e9f2342238d453cf268b1077e/artifacts/model \
#     -n my-docker-image \
#     --enable-mlserver

In [25]:
pwd

'/Users/macbook/Documents/GitHub/mlflow/wine_quality'

In [29]:
!mlflow models serve -m $ARTIFACT_STORE -h $SERVER_HOST -p $SERVER_PORT --no-conda

Usage: mlflow models serve [OPTIONS]
Try 'mlflow models serve --help' for help.

Error: Invalid value for '--port' / '-p': '--no-conda' is not a valid integer.


In [30]:
import docker
